# Running a Federated Cycle with Synergos
This notebook shows how to use Synergos to run a federated cycle through three phases: Connect, Train, Evaluate, for the [Imagenette dataset](https://github.com/fastai/imagenette).

### About the dataset:
* A subset of classes from Imagenet

* This demo will perform classification on a subset of Imagenette with 3 classes 

### Prerequisites:
Before running this notebook, you should have:
* Built the required docker images for TTP and Worker
* Installed Synergos in your (virtual) environment

## 1. Data preprocessing

The image data has been resized to 28*28 and placed into directories for workers 

In [ ]:
# ./imagenette # contains the datasets in this demo ready to be distributed/mounted to the worker containers  
#     /data[n] # for each worker participating  
#         /train  
#             metadata.json  
#             /classX
#                 img1.jpg
#                 img2.jpg
#             /classY
#                 img3.jpg
#                 img4.jpg
#             /classZ
#                 img5.jpg
#                 img6.jpg
#         /evaluate
#             metadata.json  
#             /classX
#                 img1.jpg
#                 img2.jpg
#      ... and so on

## 2. Set up TTP and worker nodes

#### Running on local machine with local participants:

Execute these commands from within your working project directory, in separate terminals

````
docker run -v "$(pwd)"/imagenette/data1:/worker/data -v "$(pwd)"/demo_outputs/worker1:/worker/outputs --name worker_1 worker:pysyft_demo

docker run -v "$(pwd)"/imagenette/data2:/worker/data -v "$(pwd)"/demo_outputs/worker2:/worker/outputs --name worker_2 worker:pysyft_demo

docker run -p 5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v "$(pwd)"/demo_outputs:/ttp/mlflow -v "$(pwd)"/imagenette/ttp_data:/ttp/data -v "$(pwd)"/demo_outputs/ttp:/ttp/outputs --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo
````


In [1]:
from synergos import Driver

host = "0.0.0.0"    # IP and port of TTP service
port = 5000

driver = Driver(host=host, port=port)

############################################################
# Phase 1: CONNECT - Submitting TTP & Participant metadata #
############################################################

# 1A. TTP controller creates a project

driver.projects.create(
    project_id="test_project",
    action="classify",     # either regress or classify
    incentives={
        'tier_1': [],
        'tier_2': [],
        'tier_3': []
    }
)


# 1B. TTP controller creates an experiment

# Define a simple two layer neural net
driver.experiments.create(
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        # Input: N, C, Height, Width [N, 1, 28, 28]
        {
            "activation": "relu",
            "is_input": True,
            "l_type": "Conv2d",
            "structure": {
                "in_channels": 1, 
                "out_channels": 4, # [N, 4, 28, 28]
                "kernel_size": 3,
                "stride": 1,
                "padding": 1
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Flatten",
            "structure": {}
        },
        {
            "activation": "sigmoid",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 4 * 28 * 28,
                "out_features": 3
            }
        }
    ]
)


# 1C. TTP controller creates a run

driver.runs.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2,
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)


# 1D. Participants registers their servers' configurations

driver.participants.create(
    participant_id="test_participant_1",
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)


# 1E. Participants registers their role in a specific project
# Roles: Host: contribute data # Guest: has validation set, may also contribute data.

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_1",
    role="guest"              
)

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_2",
    role="host"
)


# 1F. Participants registers their tags for a specific project

driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_1",
    train=[["train"]],
    evaluate=[["evaluate"]],
)

driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_2",
    train=[["train"]],
    evaluate=[["evaluate"]]
)




{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'train': [['train']],
  'evaluate': [['evaluate']],
  'predict': []},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'tags.post',
 'params': {'project_id': 'test_project',
  'participant_id': 'test_participant_2'}}

In [2]:
#######################################################
# Phase 2: TRAIN - Alignment, Training & Optimisation #
#######################################################

# 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

driver.alignments.create(project_id="test_project")

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'project_id': 'test_project'},
 'data': [{'created_at': '2020-09-29 03:42:11 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project'},
   'link': {'alignment_id': 'c200b958020511ebabe60242ac110004',
    'registration_id': 'ab77567e020511ebae990242ac110004',
    'tag_id': 'ab8a9630020511eb887b0242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 1,
   'kind': 'Alignment'},
  {'created_at': '2020-09-29 03:42:11 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'test_project'},
   'link': {'alignment_id': 'c20d470e020511ebabe60242ac110004',
    'registration_id': 'ab8279be020511eb9dc60242ac110004',
    'tag_id': 'ab9a00fc020511eb96ec0242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 2,
   'kind': 'Alignment'}]}

In [3]:
# 2B. Trigger training across the federated grid

model_resp = driver.models.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

In [4]:
################################################
# Phase 3: EVALUATE - Validation & Predictions #
################################################

# 3A. Perform validation(s) of combination(s)

driver.validations.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'message': 'Internal Server Error'}

In [5]:

# 3B. Perform prediction(s) of combination(s)

driver.predictions.create(
    tags={"test_project": [["predict"]]},
    participant_id="test_participant_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'predict': {'statistics': {'accuracy': [0.6410256410256411,
      0.6666666666666666,
      0.358974358974359],
     'roc_auc_score': [0.5314285714285715,
      0.5339506172839507,
      0.5946745562130178],
     'pr_auc_score': [0.41838704490828627,
      0.3456942905925999,
      0.4973040083636203],
     'f_score': [0.0, 0.13333333333333333, 0.489795918367347],
     'TPRs': [0.0, 0.08333333333333333, 0.9230769230769231],
     'TNRs': [1.0, 0.9259259259259259, 0.07692307692307693],
     'PPVs': [0.0, 0.3333333333333333, 0.3333333333333333],
     'NPVs': [0.6410256410256411, 0.6944444444444444, 0.6666666666666666],
     'FPRs': [0.0, 0.07407407407407407, 0.9230769230769231],
     'FNRs': [1.0, 0.9166666666666666, 0.07692307692307693],
     'FDRs': [0.0, 0.6666666666666666, 0.6666666666666

In [6]:
# 3B. Perform prediction(s) of combination(s)

driver.predictions.create(
    tags={"test_project": [["predict"]]},
    participant_id="test_participant_2",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '2',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'predict': {'statistics': {'accuracy': [0.6410256410256411,
      0.6666666666666666,
      0.358974358974359],
     'roc_auc_score': [0.5314285714285715,
      0.5339506172839507,
      0.5946745562130178],
     'pr_auc_score': [0.41838704490828627,
      0.3456942905925999,
      0.4973040083636203],
     'f_score': [0.0, 0.13333333333333333, 0.489795918367347],
     'TPRs': [0.0, 0.08333333333333333, 0.9230769230769231],
     'TNRs': [1.0, 0.9259259259259259, 0.07692307692307693],
     'PPVs': [0.0, 0.3333333333333333, 0.3333333333333333],
     'NPVs': [0.6410256410256411, 0.6944444444444444, 0.6666666666666666],
     'FPRs': [0.0, 0.07407407407407407, 0.9230769230769231],
     'FNRs': [1.0, 0.9166666666666666, 0.07692307692307693],
     'FDRs': [0.0, 0.6666666666666666, 0.6666666666666